## 1. 스레드와 코루틴 비교

### spinner example
CMD에서 해야 정상작동

In [5]:
import sys, time
import multiprocessing

DELAY = 0.1
DISPLAY = [ '|', '/', '-', '\\' ]

def spinner_func(before='', after=''):
    write = sys.stdout.write
    flush = sys.stdout.flush
    pos = -1
    while True:
        pos = (pos + 1) % len(DISPLAY)
        msg = before + DISPLAY[pos] + after
        write(msg); flush()
        write('\x08' * len(msg))
        time.sleep(DELAY)

def long_computation():
    # emulate a long computation
    time.sleep(3)

if __name__ == '__main__':
    spinner = multiprocessing.Process(
        None, spinner_func, args=('Please wait ... ', ''))
    spinner.start()
    try:
        long_computation()
        print('Computation done')
    finally:
        spinner.terminate()

Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... Please wait ... /Computation done


*********

### spinner_thread
CMD 창에서 해야 정상작동

In [2]:
import threading
import itertools
import time
import sys

In [3]:
class Signal:
    go = True
# 이 클래스는 외부에서 스레드를 제어하기 위해 사용할 
# go 속성 하나만 있는 간단한 가변 객체를 정의

In [4]:
#이 함수는 별도의 스레드에서 실행, signal 인자는 Signal 클래스의 객체를 받음
def spin(msg, signal):
    write = sys.stdout.write
    flush = sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        #itertools.cycle()은 주어진 시퀀스를 순환하면서 끝없이 항목을 생성
        # 사실상 무한루프
        
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        # 텍스트 모드 애니메이션 기법, 
        # 문자열 길이만큼 백스테이스(\x08)를 반복해서 커서를 앞으로 이동
        
        time.sleep(.1)
        
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status))
    # 공백 문자로 덮어쓰고 다시 커서를 처음으로 이동해서 메시지 출력행을 clean    

In [5]:
def slow_function():
    #입출력을 위해 장시간 기다리는 것처럼 보이게 만듦
    
    time.sleep(3) 
    # 주 스레드에서 sleep( ) 함수를 호출할 때 GIL이 해제, 두번째 스레드 진행
    
    return 42 

In [6]:
def supervisor():
    #두번째 스레드를 만들고, 스레드 객체를 출력, 시간이 오래 걸리는 연산 수행 후 스레드 제거
    
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('Thinking!', signal))
    print('spinner object:', spinner) # 두번째 스레드 객체 출력
    
    spinner.start() # 두번째 스레드 실행
    result = slow_function() 
    # 이 함수가 실행되면, 주 스레드가 블로킹 됨
    # 그리고 두번째 스레드가 텍스트 애니메이션 출력
    
    signal.go =False # spin 안의 for loop 중단
    spinner.join() # spinner 스레드가 끝날 때까지 기다림
    
    return result

In [7]:
def main():
    result = supervisor()
    print('Answer: ', result)
    
if __name__ == '__main__':
    main()

# 파이썬에는 스레드를 종료시키는 api가 정의되어 있지 않음
# 스레드에 메시지를 보내서 종료해야하는데 여기서는 signal.go가 그 역할을 함

spinner object: <Thread(Thread-4, initial)>
| Thinking/ Thinking- Thinking\ Thinking| Thinking/ Thinking- Thinking\ Thinking| Thinking/ Thinking- Thinking\ Thinking| Thinking/ Thinking- Thinking\ Thinking| Thinking/ Thinking- Thinking\ Thinking| Thinking/ Thinking- Thinking\ Thinking| Thinking/ Thinking- Thinking\ Thinking| Thinking/ Thinking          Answer:  42


***********

### spinner_async

In [8]:
import asyncio
import itertools
import sys

In [9]:
@asyncio.coroutine
def spin(msg):
# 스레드를 종료하기 위해 사용했던 signal 인수 필요 없음
    write = sys.stdout.write
    flush = sys.stdout.flush
    
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            yield from asyncio.sleep(.1)
            # 이벤트 루프를 블로킹하지 않고 잠자기 위해 time.sleep대신에 yield~.sleep 사용
        except asyncio.CancelledError:
        #spin()이 깨어난 후 예외가 발생하면, 취소가 요청된 것 -> 루프 종료
            break
    write(' ' * len(status) + '\x08' * len(status))

In [10]:
@asyncio.coroutine
def slow_function():
# slow_function은 이제 코루틴으로서, 
# 코루틴이 잠자면서 입출력을 수행하는 체 하는 동안 이벤트 루프가 진행될 수 있게 하기 위해
#yield from 사용

    yield from asyncio.sleep(3)
    # 메인루프의 제어흐름을 처리, 메인루프는 잠자고 난 후 코루틴을 계속 실행
    return 42

In [11]:
@asyncio.coroutine
def supervisor():
# 코루틴, yield from 을 사용해서 slow_function을 사용할 수 있음
    spinner = asyncio.async(spin('thinking!'))
    # asyncio.async는 spin() 코루틴의 실행을 스케줄링, task 객체 안에 넣어 task 객체 즉시 반환
    print('spinner object: ', spinner)
    # task 객체 출력
    result = yield from slow_function()
    # slow_function 함수를 구동해서 실행이 완료되면 반환된 값 가져옴
    # 이벤트 루프 게속 실행
    # slow_function이 yield from asyncio.sleep을 실행해서 메인루프로 제어권 넘김
    spinner.cancel()
    # task 객체는 cancel 메서드를 호출해서 취소할 수 있음
    # 그러면 예외 발생, 코루틴은 예외를 잡아서 지연시키거나 취소 요청을 거부
    return result

In [13]:
def main():
    loop = asyncio.get_event_loop()
    # 이벤트 루프에 대한 참조 가져옴
    result = loop.run_until_complete(supervisor())
    # supervisor 코루틴을 구동해서 완료
    # 코루틴의 반환값은 run_until_complete의 반환값이 됨
    loop.close()
    print('Answer: ', result)
    
if __name__ == '__main__':
    main()



RuntimeError: This event loop is already running

spinner object:  <Task pending coro=<spin() running at <ipython-input-9-0f8f91d59294>:1>>
| thinking\ thinking/ thinking| thinking!

/home/inisoft/anaconda3/envs/dev_yehoon/lib/python3.5/site-packages/ipykernel_launcher.py:3: DeprecationWarning: asyncio.async() function is deprecated, use ensure_future()
  This is separate from the ipykernel package so we can avoid doing imports until


- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinking| thinking\ thinking/ thinking| thinking- thinking/ thinking\ thinking- thinkin